In [1]:
import csv, psycopg2

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=nebula_ipfs_2022_05_03 user=postgres")
cur = conn.cursor()

In [3]:
cur.execute("select id,multi_hash from peers;")
peerids = {x[0]:x[1] for x in cur.fetchall()} # nebulaID -> peerID

In [4]:
with open("data/all-peerids.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerows([(p, peerids[p]) for p in peerids])

In [5]:
cur.execute("select distinct(crawl_id) from neighbors;")
crawl_ids=[c[0] for c in cur.fetchall()]

In [6]:
print(len(crawl_ids))
print(max(crawl_ids))

3522
3525


In [7]:
for i in range (1, max(crawl_ids)):
    if i not in crawl_ids:
        print(i)

1729
3521
3524


In [8]:
def store_crawls_in_csv(crawl_ids, filename):
    nebula_peers = []
    for n in crawl_ids:
        cur.execute("select peer_id,neighbor_ids from neighbors where crawl_id="+str(n)+";")
        neighbors_rel = cur.fetchall()
        nebula_peers += [[n, nebula_id, peerids[nebula_id]] + (neighbors if neighbors is not None else []) for (nebula_id, neighbors) in neighbors_rel]
        
    #with open(filename, 'w'):
    f = open(filename, 'w')
    writer = csv.writer(f)
    writer.writerows(nebula_peers)
        
    f.close()

In [48]:
r = [3518 + i for i in range(2)] # 2 crawls
store_crawls_in_csv(r, "data/nebula-peers-2crawls.csv")

In [47]:
r = [3515 + i for i in range(5)] # 5 crawls
store_crawls_in_csv(r, "data/nebula-peers-5crawls.csv")

In [9]:
r = [3000 + 12*i for i in range(28)] # 7 days, 4 crawls per day = 28 crawls (taken every 6 hours, 1 every 12 crawls)
store_crawls_in_csv(r, "data/nebula-peers-1week.csv")